In [50]:
import pickle 

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [51]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [52]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))


### 평가

### Baseline (w/o RAG)

In [53]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]


In [54]:
predictions

['글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:\n\n1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 이는 소비와 투자를 촉진하기 위한 조치입니다.\n\n2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 증가하고, 이는 자본 공급을 늘려 금리를 낮추는 요인으로 작용합니다.\n\n3. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추어 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.\n\n4. **글로벌화**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 자본 시장의 통합이 이루어졌습니다. 이는 금리의 국제적 동조화를 촉진하고, 전 세계적으로 금리가 낮아지는 현상을 초래했습니다.\n\n5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 주요 중앙은행들은 양적 완화(QE)와 같은 비전통적 통화 정책을 통해 대규모로 자산을 매입하고, 금리를 낮추는 정책을 지속적으로 시행해 왔습니다.\n\n6. **저물가**: 전 세계적으로 물가 상승률이 낮게 유지되면서, 중앙은행들은 인플레이션 목표를 달성하기 위해 금리를 낮게 유지하는 경향이 있습니다.\n\n이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.',
 '고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높은 위험을 감수하는 투자 상품이나 자산에 집중하는 현상을 말합니다. 이러한 현상은 여러 가지 요인에 의해 발생할 수 있습니다.\n\n1. **저금리 환경**: 중앙은행의 저금리 정책으로 인해 안전한 자산(예: 국채, 예금)의 수익률이 낮아지면, 투자자들은 더 높은 수익을 추구하기 위해 고위험 자산으로 눈을 돌리게 됩니다.\n\n2. **수익률 추구**: 투자자들은 더 높은 수익을 기대하며 주식, 고수익

In [56]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

results = evaluate(dataset, metrics=[factual_correctness_metric])

results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:35<00:00,  3.59s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다...   
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행(Investment Banks, IB)들은 다양한...   
4  금융시스템 개혁법은 국가마다 다를 수 있습니다. 예를 들어, 미국의 경우 2010년...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.25  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.22  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.05  
3  2000년대 이후 글로벌 IB들은 

In [57]:
print(results)

{'factual_correctness': 0.1320}


In [58]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [59]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 이는 소비와 투자를 촉진하기 위한 조치입니다.

2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 증가하고, 이는 자본 공급을 늘려 금리를 낮추는 요인으로 작용합니다.

3. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추어 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.

4. **글로벌화**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 자본 시장의 통합이 이루어졌습니다. 이는 금리의 국제적 동조화를 촉진하고, 전 세계적으로 금리가 낮아지는 현상을 초래했습니다.

5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 주요 중앙은행들은 양적 완화(QE)와 같은 비전통적 통화 정책을 통해 대규모로 자산을 매입하고, 금리를 낮추는 정책을 지속적으로 시행해 왔습니다.

6. **저물가**: 전 세계적으로 물가 상승률이 낮게 유지되면서, 중앙은행들은 인플레이션 목표를 달성하기 위해 금리를 낮게 유지하는 경향이 있습니다.

이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.


In [60]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:54<00:00,  5.48s/it]


In [ ]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

results = evaluate(dataset, metrics=[factual_correctness_metric])

results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:25<00:00,  2.60s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 ...   
1  고수익-고위험 부문으로의 쏠림현상은 저금리 기조 하에서 투자자들이 원하는 수익률을 ...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행들(IB)은 여러 전략을 추진하였습니다. 주요 전략...   
4  금융시스템 개혁법에 대한 정보는 주어진 문맥에서 명확하게 언급되지 않았습니다. 그러...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.53  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.55  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.62  
3  2000년대 이후 글로벌 IB들은 

In [63]:
print(results)

{'factual_correctness': 0.5580}


In [64]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이러한 정책은 금리를 낮추고 유동성을 공급하여 경제를 부양하려는 목적이 있었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태로 인해 경제가 큰 타격을 받았고, 이를 극복하기 위해 각국 중앙은행들이 다시 한 번 완화적 통화정책을 강화하였습니다. 이로 인해 저금리 기조가 더욱 부각되었습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


In [65]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
